## 1st Step
### Tokenizing the Entire story
### Story used :- *The Verdict*

In [30]:
with open("the-verdict.txt","r" , encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### print(raw_text[:99]) is use to print first 100 characters of the sentence

### *We are using Regular Expresion or re library to splitting the data into tokens*

In [31]:
import re
text = "Hello, world. This is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [32]:
result = re.split(r'([,.]|\s)', text) ## We are adding . and , as diffrent tokken
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [33]:
result = [item for item in result if item.strip()] ## Removing the witespaces
print(result)

['Hello', ',', 'world', '.', 'This', 'is', 'a', 'test', '.']


### Removing the witespaces is not always good as identation is also important in training the LLM . But here we are removing it to save spave and reduce the computation power

In [34]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [35]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [36]:
print(len(preprocessed))

4690


## 2nd Step
### Converting tokens into token id's
Vocabulary are created during converting tokens into token id's. In this vocabulary words are given number according to the alphabetical order . This number or integer is known as tokenID

In [37]:
all_word = sorted(set(preprocessed))
vocab_size = len(all_word)

print(vocab_size)

1130


In [38]:
vocab ={token:integer for integer,token in enumerate(all_word)}

In [39]:
for i,item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


### str_to_int use dictionary [vocab] to convert str to int and
### int_to_str use reverse_dictionary to convert int to str

In [40]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()
                       ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [41]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
            Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [42]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [43]:
# What if we use data which is not in story
#text = "Hello, do you like tea?"
#print(tokenizer.encode(text))

### Dealing with Special Context Tokens
We will modify the tokenizer to handel the unknowns words.
### Unknown and end_of_text

In [44]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [45]:
len(vocab.items())

1132

In [46]:
 for i,item in enumerate(list(vocab.items())[-5:]):
     print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [47]:
class SimpleTokenizerV2:

    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?!:;_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                       else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [48]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [49]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [50]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## There are also other tokens such as the following
### [BOS] (beginning of sequence):
The token mark the start of a text.
### [EOS] (end of sequence):
The token is positioned at the end of a text.
### [PAD] (padding):
while training LLMs with batch size larger than one the batch might contain texts of varying lengths.

## BYTE PAIR ENCODING (BPE)
In GPT model they don't use unknown they use BYTE PAIR ENCODING, in which the words are broken into subwords and these subwords will become tokens.

## Tokenizers Types
1. Word based tokenizer ['My','name','snehal'] Problem-OOV words.
2. Character based tokenizer ['M','y','n','a','m','e','S','n','e','h','a','l'] No OOV words problem, meaning will be lost , too many tokens
3. Sub word based tokenizer
   ### Rules -
   1. Do not split frequentlu used words into smaller subwords.
   2. Split the rare word into smaller meaningful subwords.
      
      eg - "boy" should not be split
           "boys" should be split into "boy" and "s"

BYTE PAIR ALGO IS ALSO A SUB WORD TOKENIZER
   

In [51]:
import importlib
import tiktoken

print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.9.0


In [52]:
tokenizer = tiktoken.get_encoding("gpt2")

In [53]:
text = (
    "Hello do you know me? <|endoftext|> I don't knowyou."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 466, 345, 760, 502, 30, 220, 50256, 314, 836, 470, 760, 5832, 13]


In [54]:
strings = tokenizer.decode(integers)
print(strings)

Hello do you know me? <|endoftext|> I don't knowyou.


In [55]:
integers = tokenizer.encode("Akwifns dfs")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 361, 5907, 288, 9501]
Akwifns dfs


In [56]:
integers = tokenizer.encode("Megamasterfull world with no meaning have zero sense")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[42672, 321, 1603, 12853, 995, 351, 645, 3616, 423, 6632, 2565]
Megamasterfull world with no meaning have zero sense


## Creating Input - Target pairs
Autoregresive

In [57]:
with open("the-verdict.txt","r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [58]:
enc_sample = enc_text[50:]

In [59]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [60]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context,"--->",desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


Everything left of the arrow (--->) refers to the input an LLM would receive, and the tokenID on the right side of the arrow is the target ID that the LLM will predict

In [61]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context),"--->",tokenizer.decode([desired]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


We have to create two tensors one input tensors and one output tensors
### Dataset and DataLoader

In [62]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [63]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                        stride=128, shuffle=True, drop_last=True,
                        num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [64]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [65]:
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.6.0+cu124
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [66]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


## CREATING TOKEN EMBEDDINGS

In [67]:
input_ids = torch.tensor([2, 3, 5, 1])

### We are using the embedding size of 3

In [68]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [69]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


### These weights are random they are normalized during LLM training

In [70]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [71]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [73]:
tdx = torch.tensor([2, 3, 1])

In [75]:
vocab_size = 4
output_dim = 5
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [80]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)


In [82]:
print(embedding_layer(tdx))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)
